<a href="https://colab.research.google.com/github/NayibGS07/Diplomado_Ciencia_de_Datos/blob/main/Diplomado_Ciencia_de_Datos_2024/Ciencia_de_Datos_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DIPLOMADO CIENCIA MATEMÁTICA DE DATOS

## MODULO: SISTEMAS DE RECOMENDACIÓN

## Libreries

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from scipy import spatial
import operator

## Data Loading

In [8]:
movies_df = pd.read_csv('/content/movies.csv')
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [9]:
ratings_df = pd.read_csv('/content/ratings.csv')
ratings_df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


## Data Preparation

In [10]:
movies_df['title'] = movies_df.title.str[:-7]
movies_df.head()

,movieId,title,genres
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy


In [11]:
movies_df['genres'] = ([genre.split("|") for genre in movies_df['genres']])
gen = []
for genre in movies_df['genres']:
    gen.extend(genre)
gen = list(set(gen))
gen.sort()
gen_vec = np.zeros(len(gen), dtype = int)
genre = []
for genres in movies_df['genres']:
    gv = np.zeros(len(gen), dtype = int)
    for i in range(len(genres)):
        for j in range(len(gen)):
            if genres[i] == gen[j]:
                gv[j] += 1
                gen_vec[j] += 1
                break
    genre.append(gv)
movies_df['genres_v'] = pd.Series(genre)
movies_df

,movieId,title,genres,genres_v
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]","[0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1,2,Jumanji,"[Adventure, Children, Fantasy]","[0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
2,3,Grumpier Old Men,"[Comedy, Romance]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]","[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,5,Father of the Bride Part II,[Comedy],"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,"[Action, Animation, Comedy, Fantasy]","[0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
9738,193583,No Game No Life: Zero,"[Animation, Comedy, Fantasy]","[0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
9739,193585,Flint,[Drama],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
9740,193587,Bungo Stray Dogs: Dead Apple,"[Action, Animation]","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [12]:
df = ratings_df.merge(movies_df, on = 'movieId')
df = df[['userId', 'title', 'movieId', 'rating', 'genres', 'genres_v']]
df = df.sort_values(["userId", "rating", "title"], ascending = (True, False, True)).reset_index(drop = True)
df['userId'] = df['userId'].astype('string')
df

,userId,title,movieId,rating,genres,genres_v
0,1,"Adventures of Robin Hood, The",940,5.0,"[Action, Adventure, Romance]","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,Alice in Wonderland,1032,5.0,"[Adventure, Animation, Children, Fantasy, Musi...","[0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, ..."
2,1,All Quiet on the Western Front,1927,5.0,"[Action, Drama, War]","[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,1,American Beauty,2858,5.0,"[Drama, Romance]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,1,American History X,2329,5.0,"[Crime, Drama]","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
100831,610,Who's That Knocking at My Door?,8840,1.0,[Drama],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
100832,610,X-Men Origins: Wolverine,68319,1.0,"[Action, Sci-Fi, Thriller]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
100833,610,Bloodsport: The Dark Kumite,145951,0.5,"[Action, Thriller]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
100834,610,"Crow, The: Wicked Prayer",61818,0.5,"[Action, Adventure, Fantasy, Thriller]","[0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."


## Exploratory Analysis

In [13]:
movies = pd.DataFrame(df['title'].value_counts()).reset_index()
movies.columns = ['Title', 'Freq']
movies = movies[movies['Freq']>=100]
movies

,Title,Freq
0,Forrest Gump,329
1,"Shawshank Redemption, The",317
2,Pulp Fiction,307
3,"Silence of the Lambs, The",279
4,"Matrix, The",278
...,...,...
134,Cliffhanger,101
135,Austin Powers: International Man of Mystery,100
136,Cast Away,100
137,V for Vendetta,100


In [91]:
top_n = 10
fig = px.bar(movies.head(top_n), x='Title', y='Freq', color = 'Title', text = 'Freq',
             title = 'Top ' + str(top_n) + ' - Most Viewed Movies', template = 'plotly_white')
fig.update_layout(showlegend=False)
fig.show()

In [17]:
df = df[df['title'].isin(list(movies['Title'].unique()))]
users = pd.DataFrame(df['userId'].value_counts()).reset_index()
users.columns = ['UserId', 'Freq']
users

,UserId,Freq
0,414,139
1,599,135
2,68,135
3,480,130
4,274,126
...,...,...
592,138,1
593,55,1
594,481,1
595,3,1


In [92]:
top_n = 10
fig = px.bar(users.head(top_n), x='UserId', y='Freq', color = 'UserId', text = 'Freq',
             title = 'Top ' + str(top_n) + ' - Most Active Users', template = 'plotly_white')
fig.update_layout(showlegend=False)
fig.show()

In [93]:
fig = px.histogram(users, x = 'Freq', marginal = 'violin', template = 'plotly_white', title = 'Histogram - Movies viewed by Users')
fig.update_layout(bargap=0.2)
fig.show()

In [66]:
users['Freq'].describe()

count        597.0
mean     34.048576
std      28.674042
min            1.0
25%           13.0
50%           26.0
75%           46.0
max          139.0
Name: Freq, dtype: Float64

In [67]:
movies_ratings = df[['title', 'rating']].groupby(by=["title"]).agg({'title':'size', 'rating':'mean'})
movies_ratings.columns = ['Freq', 'Rating']
movies_ratings = movies_ratings.reset_index()
movies_ratings = movies_ratings.sort_values(["Rating", "Freq", "title"], ascending = (False, False, True)).reset_index(drop = True)
movies_ratings

,title,Freq,Rating
0,"Shawshank Redemption, The",317,4.429022
1,"Godfather, The",192,4.289062
2,Fight Club,218,4.272936
3,"Godfather: Part II, The",129,4.259690
4,"Departed, The",107,4.252336
...,...,...,...
134,"Net, The",112,3.040179
135,Cliffhanger,101,3.034653
136,Home Alone,116,2.995690
137,Batman Forever,137,2.916058


In [94]:
fig = px.histogram(movies_ratings, x = 'Rating', marginal = 'box', nbins = 10, template = 'plotly_white', title = 'Histograma - Movie Ratings')
fig.update_layout(bargap=0.2)
fig.show()

In [95]:
fig = px.bar(x=gen, y=gen_vec, color = gen, text = gen_vec,
             title = 'Movie Genres', template = 'plotly_white')
fig.update_layout(showlegend=False)
fig.show()

## Recomendation System - User Based

In [29]:
ratings = df[['userId', 'movieId', 'title', 'rating']]
ratings = ratings.merge(users, left_on = 'userId', right_on = 'UserId')
ratings = ratings.drop(['UserId'], axis = 1)
#ratings = ratings[(ratings['Freq'] >= 200) & (ratings['Freq'] <= 500)]
ratings

,userId,movieId,title,rating,Freq
0,1,2858,American Beauty,5.0,58
1,1,2329,American History X,5.0,58
2,1,1517,Austin Powers: International Man of Mystery,5.0,58
3,1,1270,Back to the Future,5.0,58
4,1,1732,"Big Lebowski, The",5.0,58
...,...,...,...,...,...
20322,610,318,"Shawshank Redemption, The",3.0,98
20323,610,2628,Star Wars: Episode I - The Phantom Menace,3.0,98
20324,610,1721,Titanic,3.0,98
20325,610,3147,"Green Mile, The",2.5,98


In [40]:
userRatings = ratings.pivot_table(index = ['userId'], columns = ['title'], values = 'rating').reset_index(drop = True)
userRatings

title,2001: A Space Odyssey,Ace Ventura: Pet Detective,Aladdin,Alien,Aliens,"Amelie (Fabuleux destin d'Amélie Poulain, Le)",American Beauty,American History X,American Pie,Apocalypse Now,...,"Truman Show, The",Twelve Monkeys (a.k.a. 12 Monkeys),Twister,Up,"Usual Suspects, The",V for Vendetta,WALL·E,Waterworld,Willy Wonka & the Chocolate Factory,X-Men
0,NaN,NaN,NaN,4.0,NaN,NaN,5.0,5.0,NaN,4.0,...,NaN,NaN,3.0,NaN,5.0,NaN,NaN,NaN,5.0,5.0
1,NaN,NaN,4.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
4,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,5.0,NaN,NaN,4.0,4.5,5.0,NaN,NaN,4.0,5.0,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
593,NaN,NaN,NaN,5.0,5.0,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
595,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
corrMatrix = userRatings.corr(method='pearson', min_periods=200)
corrMatrix.head()

title,2001: A Space Odyssey,Ace Ventura: Pet Detective,Aladdin,Alien,Aliens,"Amelie (Fabuleux destin d'Amélie Poulain, Le)",American Beauty,American History X,American Pie,Apocalypse Now,...,"Truman Show, The",Twelve Monkeys (a.k.a. 12 Monkeys),Twister,Up,"Usual Suspects, The",V for Vendetta,WALL·E,Waterworld,Willy Wonka & the Chocolate Factory,X-Men
title,,,,,,,,,,,,,,,,,,,,,
2001: A Space Odyssey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ace Ventura: Pet Detective,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aladdin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alien,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aliens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
def Recomendacion_Usuario(user_id):
    myRatings = userRatings.loc[user_id].dropna()
    myRatings.sort_values(inplace = True, ascending = False)
    print('\n-> Usuario {} estas son algunas de las películas que más te han gustado:'.format(user_id))
    for i in range(10):
        print(" {}. {}".format(i+1, myRatings.index[i]))
    print('\n-> Estoy buscando las mejores recomendaciones para ti, no tardaré...')
    simCandidates = pd.Series(dtype = 'float64')
    for i in range(0, len(myRatings.index)):
        sims = corrMatrix[myRatings.index[i]].dropna()
        sims = sims.map(lambda x: x * myRatings[i])
        simCandidates = pd.concat([simCandidates, sims])
    simCandidates.sort_values(inplace = True, ascending = False)
    simCandidates = simCandidates.groupby(simCandidates.index).sum()
    simCandidates.sort_values(inplace = True, ascending = False)
    filteredSims = simCandidates.drop(myRatings.index, errors = 'ignore')
    print('\n-> Gracias por esperar, estas son unas películas que creo te pueden gustar:')
    if (len(filteredSims) < 10):
      for i in range(len(filteredSims)):
        print(" {}. {}".format(i+1, filteredSims.index[i]))
    else:
      for i in range(10):
        print(" {}. {}".format(i+1, filteredSims.index[i]))

In [77]:
user_id = input('¡Bienvenido!\nPor favor ingresa tu número de usuario: ')
Recomendacion_Usuario(int(user_id))

¡Bienvenido!
Por favor ingresa tu número de usuario: 1

-> Usuario 1 estas son algunas de las películas que más te han gustado:
 1. Batman Begins
 2. Shrek
 3. Dark Knight, The
 4. Aladdin
 5. Gladiator
 6. Lord of the Rings: The Two Towers, The
 7. Lord of the Rings: The Return of the King, The
 8. Lord of the Rings: The Fellowship of the Ring, The
 9. Up
 10. Casablanca

-> Estoy buscando las mejores recomendaciones para ti, no tardaré...

-> Gracias por esperar, estas son unas películas que creo te pueden gustar:
 1. Shawshank Redemption, The
 2. Silence of the Lambs, The



## Recommendation Systems - Item based

In [78]:
movieProp = df.groupby('movieId').agg({'rating': [np.size, np.mean]})
movieFreq = pd.DataFrame(movieProp['rating']['size'])
movieFreqNorm = movieFreq.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
moviesRatings = movies_df.merge(movieFreqNorm.reset_index(), on = 'movieId')
movieProp = movieProp.reset_index()
movieProp.columns = ['movieId', 'Freq', 'Rating']
moviesRatings = moviesRatings.merge(movieProp, on = 'movieId')
moviesRatings

,movieId,title,genres,genres_v,size,Freq,Rating
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]","[0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0.652439,215,3.920930
1,2,Jumanji,"[Adventure, Children, Fantasy]","[0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0.332317,110,3.431818
2,6,Heat,"[Action, Crime, Thriller]","[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.307927,102,3.946078
3,10,GoldenEye,"[Action, Adventure, Thriller]","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.399390,132,3.496212
4,32,Twelve Monkeys (a.k.a. 12 Monkeys),"[Mystery, Sci-Fi, Thriller]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",0.536585,177,3.983051
...,...,...,...,...,...,...,...
143,60069,WALL·E,"[Adventure, Animation, Children, Romance, Sci-Fi]","[0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.314024,104,4.057692
144,68954,Up,"[Adventure, Animation, Children, Drama]","[0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.317073,105,4.004762
145,79132,Inception,"[Action, Crime, Drama, Mystery, Sci-Fi, Thrill...","[0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, ...",0.432927,143,4.066434
146,95624,"Firm, The",[Drama],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.000000,1,3.000000


In [79]:
pd.DataFrame(moviesRatings['title']).head()

,title
0,Toy Story
1,Jumanji
2,Heat
3,GoldenEye
4,Twelve Monkeys (a.k.a. 12 Monkeys)


In [80]:
movieDict = {}
for i in range(len(moviesRatings['movieId'])):
    movieDict[moviesRatings['movieId'][i]] = (moviesRatings['title'][i], moviesRatings['genres_v'][i],
                                              moviesRatings['size'][i], moviesRatings['Rating'][i])
movieDict

{1: ('Toy Story',
  array([0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  0.6524390243902439,
  3.9209302325581397),
 2: ('Jumanji',
  array([0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  0.3323170731707317,
  3.4318181818181817),
 6: ('Heat',
  array([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]),
  0.3079268292682927,
  3.946078431372549),
 10: ('GoldenEye',
  array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]),
  0.39939024390243905,
  3.496212121212121),
 32: ('Twelve Monkeys (a.k.a. 12 Monkeys)',
  array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0]),
  0.5365853658536586,
  3.983050847457627),
 34: ('Babe',
  array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  0.3871951219512195,
  3.65234375),
 39: ('Clueless',
  array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]),
  0.31402439024390244,
  3.293269230769231),
 47: ('Seven (a.k.a. Se7en)',
  array([0, 0, 0, 0, 0, 0,

In [81]:
def Distancia(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA, genresB)
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
    return genreDistance + popularityDistance

In [82]:
def Peliculas_Similares(movieID, K):
    distances = []
    for movie in movieDict:
        if (movie != movieID):
            dist = Distancia(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

In [84]:
def Recomendacion_Pelicula(movie_id):
    print('\n-> ¡{} es una excelente película!'.format(movieDict[movie_id][0]))
    print('\n-> Estoy buscando peliculas similares a {}, no tardaré...'.format(movieDict[movie_id][0]))
    K = 25
    avgRating = 0
    recommendations = Peliculas_Similares(movie_id, K)
    recomendaciones = []
    for recommendation in recommendations:
        avgRating += movieDict[recommendation][3]
        recomendaciones.append([movieDict[recommendation][0], movieDict[recommendation][3]])
    rec = pd.DataFrame(recomendaciones, columns = ['Title', 'Score'])
    rec = rec.sort_values(by = ['Score']).reset_index(drop = True)
    print('\n-> Gracias por esperar, estas son unas películas que creo te pueden gustar:')
    for i in range(10):
        print(" {}. {}".format(i+1, rec['Title'][i]))

In [90]:
movie_id = input('¡Bienvenido!\nPor favor ingresa una película que te gusta: ')
Recomendacion_Pelicula(int(movie_id))

¡Bienvenido!
Por favor ingresa una película que te gusta: 1

-> ¡Toy Story es una excelente película!

-> Estoy buscando peliculas similares a Toy Story, no tardaré...

-> Gracias por esperar, estas son unas películas que creo te pueden gustar:
 1. Home Alone
 2. Ace Ventura: Pet Detective
 3. Dumb & Dumber (Dumb and Dumber)
 4. Jumanji
 5. True Lies
 6. Harry Potter and the Chamber of Secrets
 7. Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone)
 8. Beauty and the Beast
 9. Pirates of the Caribbean: The Curse of the Black Pearl
 10. Aladdin
